In [ ]:
import socket

host = '192.168.1.7'
port = 8000



s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((host, port))
s.listen(5)

while True:
    conn, addr = s.accept()
    print('Conectado por', addr)
    data = conn.recv(1024)
    print('Recibido:', repr(data))
    # conn.send(b'Hola mundo')
    # conn.close()

In [ ]:
import socket
import json

# Define la dirección IP y el puerto del servidor
server_address = ('192.168.1.7', 8000)

# Crea un socket TCP
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Asocia el socket con la dirección IP y el puerto del servidor
server_socket.bind(server_address)

# Escucha por conexiones entrantes
server_socket.listen(1)

# Espera por una conexión entrante
print('Esperando por una conexión...')
client_socket, client_address = server_socket.accept()
print(f'Conexión entrante de {client_address}')

# Crea un diccionario JSON de ejemplo
data = {
    "processFinished" : True,
    "pump": 10,
    "priority" : 1,
    "rotation" : 0,
    "pulses": 25000, 
    "ka": 1,
    "kb" : 0 
    }

# Serializa el diccionario JSON a una cadena de bytes
json_data = json.dumps(data).encode('utf-8')

# Envía los datos serializados a través del socket TCP
client_socket.sendall(json_data)

while True:
    conn, addr = server_socket.accept()
    print('Conectado por', addr)
    incoming_data = conn.recv(1024)
    print('Recibido:', repr(incoming_data))

# Cierra el socket
client_socket.close()
server_socket.close()

In [4]:
import socket
import json
import time

host_name = socket.gethostname()
ip_address = socket.gethostbyname(host_name)

server_address = (ip_address, 8000)
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(server_address)
server_socket.listen(1)

pump = 24
processFinished = False

incoming_data = ''


# while incoming_data != 'pump finished':
while True:
    data = {
        "processFinished": processFinished,
        "pump": pump-1,
        "priority": 0,
        "rotation": 0,
        "pulses": 300,
        "ka": 1,
        "kb": 0,
        "primePump": False,
        "pumpProgress": 10,
        "testProgress": 50
    }
    time.sleep(1)

    print('Esperando por una conexión...')
    client_socket, client_address = server_socket.accept()
    print(f'Conexión entrante de {client_address}')
    json_data = json.dumps(data).encode('utf-8')
    client_socket.sendall(json_data)


    try:
        incoming_data = client_socket.recv(1024)
        print(f'Respuesta recibida de {client_address}: {incoming_data}')
        incoming_data = incoming_data.replace(b'\x00', b'')
        clear_incoming_data = incoming_data.decode('utf-8')
        
        try:
            # answer_decoded = incoming_data.decode()
            data = json.loads(clear_incoming_data)
            print(data)
        except socket.timeout:
            pass


        # print(incoming_data)
        if incoming_data == 'pump finished':
            # print("holito")
            # pump+=1
            incoming_data = 'x'
            break
        if pump == 5:
            processFinished = True

        if pump > 5:
            break
    except socket.timeout:
        pass


client_socket.close()
server_socket.close()


Esperando por una conexión...
Conexión entrante de ('192.168.1.8', 57077)
Respuesta recibida de ('192.168.1.8', 57077): b'{"stepFinished":true,"grams":0.209999993}\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
{'stepFinished': True, 'grams': 0.209999993}


In [2]:
import socket
import time

# Crea un socket UDP
broadcast_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Establece la opción de socket para permitir la transmisión por broadcast
broadcast_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)

# Define la dirección IP y el puerto de destino
broadcast_address = ('<broadcast>', 8000)

# Envía el mensaje por broadcast
# message = "Este es un mensaje de prueba por broadcast"
data = 'x'
while(data != 'hi sup'):

    message = "hi there"
    broadcast_socket.sendto(message.encode(), broadcast_address)

    print(f"Mensaje enviado por broadcast a {broadcast_address}")
    broadcast_socket.settimeout(1.0)

    try:
        data, server = broadcast_socket.recvfrom(4096)
        print(f'Respuesta recibida de {server}: {data}')
        data = data.decode('utf-8')
        print(data)

        if data == 'hi sup':
            break
    except socket.timeout:
        pass
broadcast_socket.close()



Mensaje enviado por broadcast a ('<broadcast>', 8000)
Respuesta recibida de ('192.168.1.8', 8000): b'hi sup'
hi sup
